# Обучение FastText

В бейзлайне предполагается использовать векторы из библиотеки FastText, которые в данном ноутбуке обучатся на домен чеков

In [13]:
from gensim.models.fasttext import FastText
import pandas as pd
import numpy as np

Выгрузим все чеки из размеченного датасета для обучения, а также из тестового датасета

In [4]:
train_df = pd.read_csv("train_supervised_dataset.csv").fillna("")
uns_train_df = pd.read_csv("train_unsupervised_dataset.csv").fillna("")
test_df = pd.read_csv("test_dataset.csv")
names = pd.concat((train_df[["name"]], test_df, uns_train_df[["name"]])).reset_index(drop=True)
names

,name,id
0,Petmax Бантик леопард с красн розой 2шт,NaN
1,87191 Бусы для елки шарики_87191,NaN
2,Футболка Piazza Italia WR011446881,NaN
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN
4,Одежда (вес) 1500,NaN
...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN


In [9]:
brands = train_df[['brand']].values

In [15]:
np.unique(brands)

array(['', '1toy', '21 век', ..., 'ячменное поле', 'яшкино', '№1 school'],
      dtype=object)

In [16]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'\b\d+\b', '', text)  # remove numbers
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    words = text.lower().split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # lemmatization and remove stop words
    return words

В качестве предобработки приведем все к нижнему регистру
Токенизируем по пробельным символам

In [17]:
names["tokens"] = names["name"].apply(preprocess_text)
names

,name,id,tokens
0,Petmax Бантик леопард с красн розой 2шт,NaN,"[petmax, бантик, леопард, красн, розой, 2шт]"
1,87191 Бусы для елки шарики_87191,NaN,"[бусы, елки, шарики_87191]"
2,Футболка Piazza Italia WR011446881,NaN,"[футболка, piazza, italia, wr011446881]"
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN,"[yi57203xone, заколка, волос, девочки]"
4,Одежда (вес) 1500,NaN,"[одежда, вес]"
...,...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN,"[f, спонж, дмакияжа, фигурный, шт]"
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN,[диффarom]
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN,"[матрас, надувной, 74см, tropical, bird, заплд..."
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN,"[пододеяльник, стм, страйп, сп, размер, 796х18..."


In [18]:
names

,name,id,tokens
0,Petmax Бантик леопард с красн розой 2шт,NaN,"[petmax, бантик, леопард, красн, розой, 2шт]"
1,87191 Бусы для елки шарики_87191,NaN,"[бусы, елки, шарики_87191]"
2,Футболка Piazza Italia WR011446881,NaN,"[футболка, piazza, italia, wr011446881]"
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN,"[yi57203xone, заколка, волос, девочки]"
4,Одежда (вес) 1500,NaN,"[одежда, вес]"
...,...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN,"[f, спонж, дмакияжа, фигурный, шт]"
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN,[диффarom]
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN,"[матрас, надувной, 74см, tropical, bird, заплд..."
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN,"[пододеяльник, стм, страйп, сп, размер, 796х18..."


In [19]:
class SentenceGenerator(object):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __iter__(self):
        for index, row in self.dataframe.iterrows():
            yield row['tokens']

sentences = SentenceGenerator(names)

Обученные векторы положим в файл `fasttext.model` для использования в моделях

In [19]:
FastText(sentences, vector_size=300, window=5, min_count=5, epochs=50).save("fasttext.model")


KeyboardInterrupt

